# Import et processing des données des différentes requêtes - partie 1 cleaning

Objectif : cleaner la raw data obtenue via Twint (située dans le dossier data/make_queries) pour nettoyer le bruit (ie. tweets non pertinent par rapport à notre problématique sur le cash ou les moyens de paiement scripturaux).
Pour l'instant, ce script ne traite que les tweets issus de la requête francophone cash et cb. 

In [1]:
from functools import reduce
import random
import re 
import time
import numpy as np
import pandas as pd
import tqdm
from datetime import date

pd.options.display.max_colwidth = None
pd.options.display.max_columns = 30

from utils import query_to_words
from utils import score_language


In [2]:
WORDS_ENGLISH = set("of will he she them they it are is be over and to can can't".split())
WORDS_FRENCH = set("de du le la les un une des il elle ils elles on nous vous moi me ma te tu toi son sa celles ceux notre nos votre vos leurs leur comment dans mais car ou et pour en par à au est sont a ont peut peuvent".split())
query_fr = '(espece OR especes OR espèce OR espèces OR billet OR billets OR piece OR pieces OR pièce OR pièces OR monnaie OR cash OR liquide OR retraits OR retrait OR distributeur OR distributeurs OR " DAB " OR guichet OR guichets OR carte OR cartes OR " CB " OR "sans contact" OR " visa " OR mastercard OR "cash back" OR " NFC " OR "Google Pay" OR "ApplePay" OR "Paylib" OR "Lydia" OR "Lyf Pay" OR "Alipay" OR "Samsung Pay" OR "Stocard Pay" OR "paiements mobiles" OR "chèques" OR chèque OR cheque OR cheques) AND (paiement OR paiements OR payer OR reglement OR reglements OR réglement OR réglements OR regler OR régler OR achat OR achats OR acheter OR retirer OR virement OR virements OR virer OR depense OR dépense OR dépenses OR depenses OR dépenser OR depenser)'
borne_temp = str(date.today()) #"2021-02-01"
print(borne_temp)

2021-05-11


## 1. Requête francophone cash

In [3]:
df = pd.read_csv(f"/home/cash/data/queries/query_fr_cash.csv")
mask_dict = {
    "billet_transport": "sncf| tgv | ter |train|avion|ratp|navigo|bus|lufthanza|easyjet|metro|gare du|gare de", # sert pour "billet" MAIS aussi pour "guichet"
    "billet_event": "foot|penalty|matchs?|psg|concerts?|arenum|tournee|billeterie|olympia|stade de france|festival|spectacle|congres",
    "vetements": "zara|la piece|vetement|piece prefere|costume|pieces? mode|pieces? vintage| taille |fringue|collection",
    "composants": "pieces? detache|pieces? rapporte|pieces? de rechange|changement de piece?|piece separe|remplacement de piece?" +
                  "|voiture|peugeot|renault|automobile|pieces? de la maison|pieces? de maison",
    "manga": "on\w piece|tomes?",
    "expression_piece": "de tte piece|de toute piece|€ piece|euros? piece|piece d'?identite",
    "crypto": "crypto|bitcoin|ethereum|eth|\w+coin|blockchain", # probablement à conserver
    "argot": "wsh|wesh|m+d+r+|p+t+d+r+|de ouf|frr+|shab|bolos+|bouf+on|hagar|zebi|frerot?",
    "expression_retrait": "retrait magasin|retrait en magasin|retrait express|retrait de commandes?|droit de retrait|retrait de points?",
    "jeux_video": "console|playstation|xbox|fortnite|micromania|iphone|ps[0-9]", # à conserver
    "misc": "liquide vaisselle|radio dab|radiodab|cash back|cash out|cash flow|comme dab|espece de|regler le probleme|air ?liquide",
}

Modification de df : colonnes 

In [38]:
# Clean la variable textuelle
df["clean_tweet"] =  df.tweet.str.lower().str.replace("\.|:|,|'|’", " ").str.replace(r'http\S+', '').str.replace(r't co+', '')

# Calcule score de pertinence des tweets : plus le score est élevé, plus le tweet est pertinent. 
words = query_to_words(query_fr)
df["pertinence_score"] = df.clean_tweet.str.count(pat="|".join(words))
df.sort_values("pertinence_score", inplace=True, ascending=False)
    
# Calcule score pour déterminer si le tweet est en anglais ou pas : si 1 ou plus, tweet anglais.  
df["french_score"] = df.clean_tweet.apply(lambda string: score_language(string,WORDS_FRENCH))
df["english_score"] = df.clean_tweet.apply(lambda string: score_language(string,WORDS_ENGLISH))
    
# Ajoute variable temporelle datetime
df["datetime"] = pd.to_datetime(df.date + " " + df.time)

Modification de df : filtrage des observations 

In [39]:
# Filtrer les tweets du dernier mois (pas encore toutes les données)
shape_avant = df.shape[0]
df = df.loc[df["date"] < borne_temp]
    
# Filtrer les tweets HS grâce aux règles métiers
df = df[df.pertinence_score > 0]
masks_series = {
    label: df.clean_tweet.str.contains(value) 
    for label, value in mask_dict.items()
}
for (label, mask) in masks_series.items():
    print(f"{label.rjust(20)}: {round(100 * mask.mean(), 1)}%")
df = df[~ reduce(lambda x, y: x | y, masks_series.values())]
    
# Filtrer les tweets autres que francophones
df = df[df.english_score < 1]
shape_inter = df.shape[0]
df = df[df.french_score>= 1]
shape_apres = df.shape[0]

print(f"Cleaning total: {shape_avant - shape_apres} lignes retirées sur {shape_avant}")
print(f"Cleaning des tweets non francophones: {shape_inter - shape_apres} lignes retirées sur {shape_inter}")

    billet_transport: 16.9%
        billet_event: 4.0%
           vetements: 0.8%
          composants: 1.1%
               manga: 1.6%
    expression_piece: 0.0%
              crypto: 2.3%
               argot: 3.7%
  expression_retrait: 0.6%
          jeux_video: 1.1%
                misc: 1.1%
Cleaning total: 114330 lignes retirées sur 314539
Cleaning des tweets non francophones: 1837 lignes retirées sur 202046


In [40]:
print(df.isnull().values.any())
print(df.clean_tweet.isnull().values.any())
df.created_at.isnull().values.any()

True
False


False

In [41]:
len(df)


200209

In [42]:
df.to_pickle(r"/home/cash/output/cleaned_queries/without_geoloc/query_fr_cash_cleaned.pickle")

In [4]:
len(pd.read_pickle(f"/home/cash/output/cleaned_queries/without_geoloc/query_fr_cash_cleaned.pickle"))

200209

## 2. Requête francophone CB 

In [7]:
df = pd.read_csv(f"/home/cash/data/queries/query_fr_cb.csv")
mask_dict = {
    "carte": "jeu de cartes?|cartes?  ?cadeaux?|carte  ?m(e|è)re|cartes? de biblioth(è|e)que|cartes?  ?graphiques?|cartes?  ?vitales?|cartes?  ?grises?|le cb|cartes? m(e|é)moires?|cartes?  ?culture|carte visa|cartes?  ?d ?identit(e|é)|cartes?  ?de  ?s(é|e)jours?|cartes?  ?blanches?|concours|cartes?  ?de  ?presse|tirages?  ?au  ?sort|cartes?  ?jaunes?|cartes?  ?sur  ?table",
    "cb" : "d(é|e)pens(er|é|e) cb"
}

In [45]:
# Clean la variable textuelle
df["clean_tweet"] =  df.tweet.str.lower().str.replace("\.|:|,|'|’", " ").str.replace(r'http\S+', '').str.replace(r't co+', '')

# Calcule score de pertinence des tweets : plus le score est élevé, plus le tweet est pertinent. 
words = query_to_words(query_fr)
df["pertinence_score"] = df.clean_tweet.str.count(pat="|".join(words))
df.sort_values("pertinence_score", inplace=True, ascending=False)
    
# Calcule score pour déterminer si le tweet est en anglais ou pas : si 1 ou plus, tweet anglais.  
df["french_score"] = df.clean_tweet.apply(lambda string: score_language(string,WORDS_FRENCH))
df["english_score"] = df.clean_tweet.apply(lambda string: score_language(string,WORDS_ENGLISH))
    
# Ajoute variable temporelle datetime
df["datetime"] = pd.to_datetime(df.date + " " + df.time)

In [46]:
# Filtrer les tweets du dernier mois (pas encore toutes les données)
shape_avant = df.shape[0]
df = df.loc[df["date"] < borne_temp]
    
# Filtrer les tweets HS grâce aux règles métiers
df = df[df.pertinence_score > 0]
masks_series = {
    label: df.clean_tweet.str.contains(value) 
    for label, value in mask_dict.items()
}
for (label, mask) in masks_series.items():
    print(f"{label.rjust(20)}: {round(100 * mask.mean(), 1)}%")
df = df[~ reduce(lambda x, y: x | y, masks_series.values())]
    
# Filtrer les tweets autres que francophones
df = df[df.english_score < 1]
shape_inter = df.shape[0]
df = df[df.french_score>= 1]
shape_apres = df.shape[0]

print(f"Cleaning total: {shape_avant - shape_apres} lignes retirées sur {shape_avant}")
print(f"Cleaning des tweets non francophones: {shape_inter - shape_apres} lignes retirées sur {shape_inter}")

/home/cash/.local/lib/python3.8/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


               carte: 11.1%
                  cb: 0.1%
Cleaning total: 38715 lignes retirées sur 294417
Cleaning des tweets non francophones: 1981 lignes retirées sur 257683


In [47]:
print(df.isnull().values.any())
print(df.clean_tweet.isnull().values.any())
df.created_at.isnull().values.any()
len(df)

True
False


255702

In [49]:
df.to_pickle(r"/home/cash/output/cleaned_queries/without_geoloc/query_fr_cb_cleaned.pickle")

with open('filename.pickle', 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('filename.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [5]:
df = pd.read_pickle("/home/cash/output/cleaned_queries/without_geoloc/query_fr_cb_cleaned.pickle")
print(len(df))

255702


## 3. Requete francophone sans-contact

In [14]:
df_contactless_cb_fr = pd.read_csv(f"/home/cash/data/queries/query_contactless_cb_fr_part1.csv")
print(df_contactless_cb_fr.shape)
print(max(df_contactless_cb_fr['date']))
df_contactless_smartphone_fr = pd.read_csv(f"/home/cash/data/queries/query_contactless_smartphone_fr_part1.csv")
print(df_contactless_smartphone_fr.shape)
print(max(df_contactless_smartphone_fr['date']))




#relancement de la requete et qu'un csv

(44459, 36)
2021-04-30
(56777, 36)
2021-04-30


In [15]:
df_contactless_smartphone_fr['type_contactless'] = 'smartphone'
df_contactless_cb_fr['type_contactless'] = 'cb'

print(df_contactless_cb_fr.shape)
print(df_contactless_smartphone_fr.shape)

(44459, 37)
(56777, 37)


In [16]:
df_contactless = pd.concat([df_contactless_cb_fr, df_contactless_smartphone_fr])
print(df_contactless.shape)

(101236, 37)


Agrégation des DFs

In [17]:
mask_dict = {
    "billet_transport": "sncf| tgv | ter |train|avion|ratp|navigo|bus|lufthanza|easyjet|metro|gare du|gare de", # sert pour "billet" MAIS aussi pour "guichet"
    "billet_event": "foot|penalty|matchs?|psg|concerts?|arenum|tournee|billeterie|olympia|stade de france|festival|spectacle|congres",
    "vetements": "zara|la piece|vetement|piece prefere|costume|pieces? mode|pieces? vintage| taille |fringue|collection",
    "composants": "pieces? detache|pieces? rapporte|pieces? de rechange|changement de piece?|piece separe|remplacement de piece?" +
                  "|voiture|peugeot|renault|automobile|pieces? de la maison|pieces? de maison",
    "manga": "on\w piece|tomes?",
    "expression_piece": "de tte piece|de toute piece|€ piece|euros? piece|piece d'?identite",
    "crypto": "crypto|bitcoin|ethereum|eth|\w+coin|blockchain", # probablement à conserver
    "argot": "wsh|wesh|m+d+r+|p+t+d+r+|de ouf|frr+|shab|bolos+|bouf+on|hagar|zebi|frerot?",
    "expression_retrait": "retrait magasin|retrait en magasin|retrait express|retrait de commandes?|droit de retrait|retrait de points?",
    "jeux_video": "console|playstation|xbox|fortnite|micromania|iphone|ps[0-9]", # à conserver
    "misc": "liquide vaisselle|radio dab|radiodab|cash back|cash out|cash flow|comme dab|espece de|regler le probleme|air ?liquide",
}

Modification des colonnes

In [18]:
# Clean la variable textuelle
df_contactless["clean_tweet"] =  df_contactless.tweet.str.lower().str.replace("\.|:|,|'|’", " ").str.replace(r'http\S+', '').str.replace(r't co+', '')

# Calcule score de pertinence des tweets : plus le score est élevé, plus le tweet est pertinent. 
words = query_to_words(query_fr)
df_contactless["pertinence_score"] = df_contactless.clean_tweet.str.count(pat="|".join(words))
df_contactless.sort_values("pertinence_score", inplace=True, ascending=False)
    
# Calcule score pour déterminer si le tweet est en anglais ou pas : si 1 ou plus, tweet anglais.  
df_contactless["french_score"] = df_contactless.clean_tweet.apply(lambda string: score_language(string,WORDS_FRENCH))
df_contactless["english_score"] = df_contactless.clean_tweet.apply(lambda string: score_language(string,WORDS_ENGLISH))
    
# Ajoute variable temporelle datetime
df_contactless["datetime"] = pd.to_datetime(df_contactless.date + " " + df_contactless.time)

Modification et filtrage

In [19]:
# Filtrer les tweets du dernier mois (pas encore toutes les données)
shape_avant = df_contactless.shape[0]
df_contactless = df_contactless.loc[df_contactless["date"] < borne_temp]

# Filtrer les tweets HS grâce aux règles métiers
df_contactless = df_contactless[df_contactless.pertinence_score > 0]
masks_series = {
    label: df_contactless.clean_tweet.str.contains(value) 
    for label, value in mask_dict.items()
}
for (label, mask) in masks_series.items():
    print(f"{label.rjust(20)}: {round(100 * mask.mean(), 1)}%")
df_contactless = df_contactless[~ reduce(lambda x, y: x | y, masks_series.values())]

    billet_transport: 3.5%
        billet_event: 0.7%
           vetements: 0.1%
          composants: 0.2%
               manga: 1.0%
    expression_piece: 0.0%
              crypto: 2.8%
               argot: 0.9%
  expression_retrait: 0.0%
          jeux_video: 2.4%
                misc: 0.1%


In [20]:
# Filtrer les tweets autres que francophones
df_contactless = df_contactless[df_contactless.english_score < 1]
shape_inter = df_contactless.shape[0]
df_contactless = df_contactless[df_contactless.french_score>= 1]
shape_apres = df_contactless.shape[0]

print(f"Cleaning total: {shape_avant - shape_apres} lignes retirées sur {shape_avant}")
print(f"Cleaning des tweets non francophones: {shape_inter - shape_apres} lignes retirées sur {shape_inter}")

Cleaning total: 53206 lignes retirées sur 101236
Cleaning des tweets non francophones: 18083 lignes retirées sur 66113


In [21]:
#print(df_disparition['created_at'].head(15))
print(df_contactless[['clean_tweet','tweet' ]].head(1))

print(df_contactless.isnull().values.any())
print(df_contactless.clean_tweet.isnull().values.any())
df_contactless.created_at.isnull().values.any()

                                                                                                                                                                                                                                                                                    clean_tweet  \
25636  @megaviria bonjour  la carte lydia est une carte de paiement mastercard conçue par lydia   comme toutes les cartes mastercard  la carte lydia est acceptée partout dans le monde  par les commerçants et les distributeurs automatiques de billets  quelle que soit la monnaie du pays !   

                                                                                                                                                                                                                                                                                          tweet  
25636  @megaviria Bonjour, La carte Lydia est une carte de paiement Mastercard conçue par Lydia.  Comme toutes les cartes Maste

False

In [22]:
df_contactless.to_pickle(r"/home/cash/output/cleaned_queries/without_geoloc/query_fr_contactless_cleaned.pickle")

In [23]:
df_contactless_lecture = pd.read_pickle(r"/home/cash/output/cleaned_queries/without_geoloc/query_fr_contactless_cleaned.pickle")
df_contactless.equals(df_contactless_lecture)

True

In [24]:
df_contactless.shape

(48030, 42)